<a href="https://colab.research.google.com/github/daljeetkaursaini/Daljeetkaursaini/blob/main/underwater_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
import numpy as np
from skimage import data, color
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal as sig
import math


In [5]:
from skimage.util import img_as_ubyte, img_as_float64
from skimage.color import rgb2gray
from skimage.color import rgb2hsv
import matplotlib.pyplot as plt
from skimage.io import imread
import warnings
warnings.filterwarnings('ignore')


In [6]:

import skimage
from numpy import load
from numpy import expand_dims
import matplotlib
from matplotlib import pyplot
import sys
import PIL
from PIL import Image


In [7]:
import pandas as pd
import numpy as np
import scipy.misc
import imageio
import glob
import os


In [8]:

import math
from math import log2, log10
def mu_a(x, alpha_L=0.1, alpha_R=0.1):
    """
      Calculates the asymetric alpha-trimmed mean
    """
    # sort pixels by intensity - for clipping
    x = sorted(x)
    # get number of pixels
    K = len(x)
    # calculate T alpha L and T alpha R
    T_a_L = math.ceil(alpha_L*K)
    T_a_R = math.floor(alpha_R*K)
    # calculate mu_alpha weight
    weight = (1/(K-T_a_L-T_a_R))
    # loop through flattened image starting at T_a_L+1 and ending at K-T_a_R
    s   = int(T_a_L+1)
    e   = int(K-T_a_R)
    val = sum(x[s:e])
    val = weight*val
    return val


In [9]:
def s_a(x, mu):
    val = 0
    for pixel in x:
        val += math.pow((pixel-mu), 2)
    return val/len(x)



In [10]:
def _uicm(x):
    R = x[:,:,0].flatten()
    G = x[:,:,1].flatten()
    B = x[:,:,2].flatten()
    RG = R-G
    YB = ((R+G)/2)-B
    mu_a_RG = mu_a(RG)
    mu_a_YB = mu_a(YB)
    s_a_RG = s_a(RG, mu_a_RG)
    s_a_YB = s_a(YB, mu_a_YB)
    l = math.sqrt( (math.pow(mu_a_RG,2)+math.pow(mu_a_YB,2)) )
    r = math.sqrt(s_a_RG+s_a_YB)
    return (-0.0268*l)+(0.1586*r)



In [11]:
def sobel(x):
    dx = ndimage.sobel(x,0)
    dy = ndimage.sobel(x,1)
    mag = np.hypot(dx, dy)
    mag *= 255.0 / np.max(mag) 
    return mag


In [12]:
def eme(x, window_size):
    """
      Enhancement measure estimation
      x.shape[0] = height
      x.shape[1] = width
    """
    # if 4 blocks, then 2x2...etc.
    k1 = x.shape[1]/window_size
    k2 = x.shape[0]/window_size
    # weight
    w = 2./(k1*k2)
    blocksize_x = window_size
    blocksize_y = window_size
    # make sure image is divisible by window_size - doesn't matter if we cut out some pixels
    x = x[:int(blocksize_y*k2), :int(blocksize_x*k1)]
    val = 0
    for l in range(int(k1)):
        for k in range(int(k2)):
            block = x[k*window_size:window_size*(k+1), l*window_size:window_size*(l+1)]
            max_ = np.max(block)
            min_ = np.min(block)
            # bound checks, can't do log(0)
            if min_ == 0.0: val += 0
            elif max_ == 0.0: val += 0
            else: val += math.log(max_/min_)
    return w*val


In [13]:
def _uism(x):
    """
      Underwater Image Sharpness Measure
    """
    # get image channels
    R = x[:,:,0]
    G = x[:,:,1]
    B = x[:,:,2]
    # first apply Sobel edge detector to each RGB component
    Rs = sobel(R)
    Gs = sobel(G)
    Bs = sobel(B)
    # multiply the edges detected for each channel by the channel itself
    R_edge_map = np.multiply(Rs, R)
    G_edge_map = np.multiply(Gs, G)
    B_edge_map = np.multiply(Bs, B)
    # get eme for each channel
    r_eme = eme(R_edge_map, 10)
    g_eme = eme(G_edge_map, 10)
    b_eme = eme(B_edge_map, 10)
    # coefficients
    lambda_r = 0.299
    lambda_g = 0.587
    lambda_b = 0.144
    return (lambda_r*r_eme) + (lambda_g*g_eme) + (lambda_b*b_eme)


In [14]:
def plip_g(x,mu=1026.0):
    return mu-x



In [15]:
def plip_theta(g1, g2, k):
    g1 = plip_g(g1)
    g2 = plip_g(g2)
    return k*((g1-g2)/(k-g2))

def plip_cross(g1, g2, gamma):
    g1 = plip_g(g1)
    g2 = plip_g(g2)
    return g1+g2-((g1*g2)/(gamma))

def plip_diag(c, g, gamma):
    g = plip_g(g)
    return gamma - (gamma * math.pow((1 - (g/gamma) ), c) )

def plip_multiplication(g1, g2):
    return plip_phiInverse(plip_phi(g1) * plip_phi(g2))
    #return plip_phiInverse(plip_phi(plip_g(g1)) * plip_phi(plip_g(g2)))

def plip_phiInverse(g):
    plip_lambda = 1026.0
    plip_beta   = 1.0
    return plip_lambda * (1 - math.pow(math.exp(-g / plip_lambda), 1 / plip_beta));

def plip_phi(g):
    plip_lambda = 1026.0
    plip_beta   = 1.0
    return -plip_lambda * math.pow(math.log(1 - g / plip_lambda), plip_beta)


In [16]:
def _uiconm(x, window_size):
    plip_lambda = 1026.0
    plip_gamma  = 1026.0
    plip_beta   = 1.0
    plip_mu     = 1026.0
    plip_k      = 1026.0
    # if 4 blocks, then 2x2...etc.
    k1 = x.shape[1]/window_size
    k2 = x.shape[0]/window_size
    # weight
    w = -1./(k1*k2)
    blocksize_x = window_size
    blocksize_y = window_size
    # make sure image is divisible by window_size - doesn't matter if we cut out some pixels
    x = x[:int(blocksize_y*k2), :int(blocksize_x*k1)]
    # entropy scale - higher helps with randomness
    alpha = 1
    val = 0
    for l in range(int(k1)):
        for k in range(int(k2)):
            block = x[k*window_size:window_size*(k+1), l*window_size:window_size*(l+1), :]
            max_ = np.max(block)
            min_ = np.min(block)
            top = max_-min_
            bot = max_+min_
            if math.isnan(top) or math.isnan(bot) or bot == 0.0 or top == 0.0: val += 0.0
            else: val += alpha*math.pow((top/bot),alpha) * math.log(top/bot)
            #try: val += plip_multiplication((top/bot),math.log(top/bot))
    return w*val


In [17]:
def getUIQM(x):
    """
      Function to return UIQM to be called from other programs
      x: image
    """
    x = x.astype(np.float32)
    ### from https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=7300447
    #c1 = 0.4680; c2 = 0.2745; c3 = 0.2576
    ### from https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=7300447
    c1 = 0.0282; c2 = 0.2953; c3 = 3.5753
    uicm   = _uicm(x)
    uism   = _uism(x)
    uiconm = _uiconm(x, 10)
    uiqm = (c1*uicm) + (c2*uism) + (c3*uiconm)
    return uiqm


In [18]:
def UCIQE(rgb_in):
    # calculate Chroma
    rgb_in = cv2.normalize(rgb_in, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    (l,a,b)=cv2.split(rgb_in)
    Chroma = np.sqrt(a*a + b*b)
    StdVarianceChroma = np.std(np.reshape(Chroma[:,:],(-1,1)))

    hsv = skimage.color.rgb2hsv(rgb_in)
    Saturation = hsv[:,:,2]
    MeanSaturation = np.mean(np.reshape(Saturation[:,:],(-1,1)))

    ContrastLuminance = max(np.reshape(l[:,:],(-1,1))) - min(np.reshape(l[:,:],(-1,1)))
    UCIQE = 0.4680 * StdVarianceChroma + 0.2745 * ContrastLuminance + 0.2576 * MeanSaturation
    return float(UCIQE)

def improve_contrast_image_using_clahe(bgr_image):
    hsv = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2HSV)
    hsv_planes = cv2.split(hsv)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    hsv_planes[2] = clahe.apply(hsv_planes[2])
    hsv = cv2.merge(hsv_planes)
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)


In [19]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy import signal as sig
import math
from skimage.util import img_as_ubyte, img_as_float64
from skimage.color import rgb2gray
from skimage.color import rgb2hsv


''' Generates guided filter''' 
def guide(I,P,r,e):

    h,w=np.shape(I)
    window = np.ones((r,r))/(r*r)

    meanI = sig.convolve2d(I, window,mode='same')
    meanP = sig.convolve2d(P, window,mode='same')

    corrI = sig.convolve2d(I*I, window,mode='same')
    corrIP = sig.convolve2d(I*P, window,mode='same')

    varI = corrI - meanI*meanI
    covIP = corrIP - meanI*meanP
    a = covIP/(varI+e)
    b = meanP - a*meanI

    meana = sig.convolve2d(a, window,mode='same')
    meanb = sig.convolve2d(b, window,mode='same')

    q = meana*I+meanb

    return q


In [20]:
def localmin(D, r=15):
    R = int(r/2)
    imax = D.shape[0]
    jmax = D.shape[1]
    LM = np.zeros([imax,jmax])
    for i in np.arange(D.shape[0]):
        for j in np.arange(D.shape[1]):
            iL = np.max([i-R,0])
            iR = np.min([i+R, imax])
            jT = np.max([j-R,0])
            jB = np.min([j+R, jmax])
            # print(D[iL:iR+1,jT:jB+1].shape)
            LM[i,j] = np.min(D[iL:iR+1,jT:jB+1])
    return LM

''' It will Apply guided filter to the images and removes haze''' 


' It will Apply guided filter to the images and removes haze'

In [21]:
def postprocessing(GD, I,V):
    # this will give indices of the columnised image GD
    flat_indices = np.argsort(GD, axis=None)
    R,C = GD.shape
    top_indices_flat = flat_indices[ int(np.round(0.999*R*C)):: ]
    top_indices = np.unravel_index(top_indices_flat, GD.shape)

    max_v_index = np.unravel_index( np.argmax(V[top_indices], axis=None), V.shape )
    I = I/255.0
    A = I[max_v_index[0], max_v_index[1], :]

    beta = 1.0
    transmission = np.minimum( np.maximum(np.exp(-1*beta*GD), 0.1) , 0.9)
    # transmission = np.exp(-1*beta*GD)
    transmission3 = np.zeros(I.shape)
    transmission3[:,:,0] = transmission
    transmission3[:,:,1] = transmission
    transmission3[:,:,2] = transmission

    J = A + (I - A)/transmission3
    J = J - np.min(J)
    J = J/np.max(J)
    return J




In [22]:

def dehaze(img):
    # Read the Image
    #img = cv2.imread("vit.jpg")
    # opencv reads any image in Blue-Green-Red(BGR) format,
    # so change it to RGB format, which is popular.
    I = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Split Image to Hue-Saturation-Value(HSV) format.
    H,S,V = cv2.split(cv2.cvtColor(img, cv2.COLOR_BGR2HSV) )
    V = V/255.0
    S = S/255.0

    # Calculating Depth Map using the linear model fit by ZHU et al.
    theta_0 = 0.121779
    theta_1 = 0.959710
    theta_2 = -0.780245
    sigma   = 0.041337
    epsilon = np.random.normal(0, sigma, H.shape )
    D = theta_0 + theta_1*V + theta_2*S + epsilon

    # Local Minima of Depth map
    LMD = localmin(D, 15)

    r = 8; # try r=2, 4, 8 or 18
    eps = 0.2 * 0.2; # try eps=0.1^2, 0.2^2, 0.4^2
        # eps *= 255 * 255;   # Because the intensity range of our images is [0, 255]
    GD=guide(D,LMD,r,eps)

    J = postprocessing(GD, I,V)
    return J


In [23]:
def clip(image=None):
    if image is None:
        return None
    image[image < 0] = 0
    image[image > 1] = 1
    return image


In [24]:
def white_balance(img):
    image = img_as_float64(img)

            # Extract colour channels
    R = image[:, :, 2]
    G = image[:, :, 1]
    B = image[:, :, 0]

            # Obtain average intensity for each colour channel
    mean_R = np.mean(R)
    mean_G = np.mean(G)
    mean_B = np.mean(B)

    mean_RGB = np.array([mean_R, mean_G, mean_B])

            # Obtain scaling factor
    grayscale = np.mean(mean_RGB)
    scale = grayscale / mean_RGB

    white_balanced = np.zeros(image.shape)

            # Rescale original intensities
    white_balanced[:, :, 2] = scale[0] * R
    white_balanced[:, :, 1] = scale[1] * G
    white_balanced[:, :, 0] = scale[2] * B

            # Clip to [0.0, 1.0]
    white_balanced = clip(white_balanced)
    return white_balanced
''' equlises the rgb chanells of the image using histograms ''' 



' equlises the rgb chanells of the image using histograms '

In [25]:
def simplest_cb(image, percent):
    if image is None:
        return None
    out_channels = []
    channels = cv2.split(image)
    totalstop = channels[0].shape[0] * channels[0].shape[1] * percent / 200.0
    for channel in channels:
        bc = cv2.calcHist([channel], [0], None, [256], (0,256), accumulate=False)
        lv = np.searchsorted(np.cumsum(bc), totalstop)
        hv = 255-np.searchsorted(np.cumsum(bc[::-1]), totalstop)
        lut = np.array([0 if i < lv else (255 if i > hv else round(float(i-lv)/float(hv-lv)*255)) for i in np.arange(0, 256)], dtype="uint8")
        out_channels.append(cv2.LUT(channel, lut))
    k = cv2.merge(out_channels)
    #k = cv2.cvtColor(k, cv2.COLOR_BGR2RGB)    
    return k




In [26]:
def luminance_map(image=None):
    if image is None:
        return None
    '''Function to generate the Luminance Weight Map of an image'''
        # Validate parameters
        
    image = img_as_float64(image)

        # Generate Luminance Map
    luminance = np.mean(image, axis=2)
    luminancemap = np.sqrt((1 / 3) * (np.square(image[:, :, 0] - luminance + np.square(image[:, :, 1] - luminance) 
                                                + np.square(image[:, :, 2] - luminance))))
    return luminancemap



In [27]:
def chromatic_map(image=None):
    if image is None:
        return None
    '''Function to generate the Chromatic Weight Map of an image'''
        # Validate parameters
        
    image = img_as_float64(image)
        
        # Convert to HSV colour space
    hsv = rgb2hsv(image)

        # Extract Saturation
    saturation = hsv[:, :, 1]
    max_saturation = 1.0
    sigma = 0.3
        
        # Generate Chromatic Map
    chromaticmap = np.exp(-1 * (((saturation - max_saturation) ** 2) / (2 * (sigma ** 2))))
    return chromaticmap



In [28]:
def saliency_map(image=None):
    if image is None:
        return None
    '''Function to generate the Saliency Weight Map of an image'''
        # Validate parameters   
    image = img_as_float64(image)
        
        # Convert image to grayscale
    if(len(image.shape) > 2):
        image = rgb2gray(image)
    else:
        image = image
        
        # Apply Gaussian Smoothing
    gaussian = cv2.GaussianBlur(image,(5, 5),0) 
        
        # Apply Mean Smoothing
    image_mean = np.mean(image)
        
        # Generate Saliency Map
    saliencymap = np.absolute(gaussian - image_mean)
    return saliencymap



In [29]:

def image_pyramid(image=None, pyramid_type='gaussian', levels=1):
    if image is None:
        return None
        '''Function to generate the Gaussian/Laplacian pyramid of an image'''
        # Validate parameters
    image = img_as_float64(image)
        
        # Generate Gaussian Pyramid
    current_layer = image
    gaussian = [current_layer]
    for i in range(levels):
        current_layer = cv2.pyrDown(current_layer)
        gaussian.append(current_layer)
            
    if pyramid_type == 'gaussian':
        return gaussian
        # Generate Laplacian Pyramid
    elif pyramid_type == 'laplacian':
        current_layer = gaussian[levels-1]
        laplacian = [current_layer]
        for i in range(levels - 1, 0, -1):
            shape = (gaussian[i-1].shape[1], gaussian[i-1].shape[0])
            expand_gaussian = cv2.pyrUp(gaussian[i], dstsize=shape)
            current_layer = cv2.subtract(gaussian[i-1], expand_gaussian)
            laplacian.append(current_layer)
        laplacian.reverse()
        return laplacian

    


In [30]:
def weight_maps(J,white_balanced):
    input_images = [
            img_as_float64(J),
            img_as_float64(white_balanced)
        ]

    weight_maps = [
                # Weight maps for first image
                {
                    'luminance': luminance_map(image=input_images[0]),
                    'chromatic': chromatic_map(image=input_images[0]),
                    'saliency': saliency_map(image=input_images[0])
                },

                # Weight maps for second image
                {
                    'luminance': luminance_map(image=input_images[1]),
                    'chromatic': chromatic_map(image=input_images[1]),
                    'saliency': saliency_map(image=input_images[1])
                }
            ]
    weight_maps[0]['combined'] = (weight_maps[0]['luminance'] * weight_maps[0]['chromatic'] * weight_maps[0]['saliency'])
    weight_maps[1]['combined'] = (weight_maps[1]['luminance'] * weight_maps[1]['chromatic'] * weight_maps[1]['saliency'])

            # Normalized weight maps
    weight_maps[0]['normalized'] = weight_maps[0]['combined'] / (weight_maps[0]['combined'] + weight_maps[1]['combined'])
    weight_maps[1]['normalized'] = weight_maps[1]['combined'] / (weight_maps[0]['combined'] + weight_maps[1]['combined'])
    
    pyramid_height=12
    gaussians = [
            image_pyramid(image=weight_maps[0]['normalized'], pyramid_type='gaussian', levels=pyramid_height),
            image_pyramid(image=weight_maps[1]['normalized'], pyramid_type='gaussian', levels=pyramid_height)
        ]
    l=[input_images,weight_maps,gaussians]
    return l




In [31]:
def fusion(inputs=None, weights=None, gaussians=None):
    if inputs is None or weights is None or gaussians is None:
        return None
        '''Function to fuse the pyramids together'''
        # Validate parameters
    fused_levels = []

        # Perform Fusion by combining the Laplacian and Gaussian pyramids
    for i in range(len(gaussians[0])):
        if len(inputs[0].shape) > 2:
            for j in range(inputs[0].shape[2]):
                laplacians = [
                        image_pyramid(image=inputs[0][:, :, j], pyramid_type='laplacian', levels=len(gaussians[0])),
                        image_pyramid(image=inputs[1][:, :, j], pyramid_type='laplacian', levels=len(gaussians[0]))
                             ]
                    # Adjust rows to match
                row_size = np.min(np.array([
                        laplacians[0][i].shape[0],
                        laplacians[1][i].shape[0],
                        gaussians[0][i].shape[0],
                        gaussians[1][i].shape[0]
                    ]))

                    # Adjust columns to match
                col_size = np.min(np.array([
                        laplacians[0][i].shape[1],
                        laplacians[1][i].shape[1],
                        gaussians[0][i].shape[1],
                        gaussians[1][i].shape[1]
                    ]))
                    
                if j == 0:
                    intermediate = np.ones(inputs[0][:row_size, :col_size].shape)
                    # Fusion Step
                intermediate[:, :, j] = (laplacians[0][i][:row_size, :col_size] * gaussians[0][i][:row_size, :col_size]) + (laplacians[1][i][:row_size, :col_size] * gaussians[1][i][:row_size, :col_size])
            fused_levels.append(intermediate)
        
        # Reconstruct Image Pyramids
    for i in range(len(fused_levels)-2, -1, -1):
        level_1 = cv2.pyrUp(fused_levels[i+1])
        level_2 = fused_levels[i]
        r = min(level_1.shape[0], level_2.shape[0])
        c = min(level_1.shape[1], level_2.shape[1])
        fused_levels[i] = level_1[:r, :c] + level_2[:r, :c]

        # Clip fused image to [0.0, 1.0]
    fused = clip(fused_levels[0])
    return fused



In [32]:
def enhance_contrast(image=None):
        '''Function to enhance contrast in an image'''
        # Validate parameters
        if image is None:
            return None

        image = img_as_float64(image)

        # Extract colour channels
        R = image[:, :, 2]
        G = image[:, :, 1]
        B = image[:, :, 0]

        # Obtain luminance using predefined scale factors
        luminance = 0.299 * R + 0.587 * G + 0.114 * B
        mean_luminance = np.mean(luminance)

        # Compute scale factor
        gamma = 2 * (0.5 + mean_luminance)

        # Scale mean-luminance subtracted colour chanels 
        enhanced = np.zeros(image.shape)
        enhanced[:, :, 2] = gamma * (R - mean_luminance)
        enhanced[:, :, 1] = gamma * (G - mean_luminance)
        enhanced[:, :, 0] = gamma * (B - mean_luminance)

        # Clip to [0.0, 1.0]
        enhanced = clip(enhanced)

        return enhanced


In [33]:
import numpy as np
import matplotlib.pyplot as plt
import imageio
import scipy, scipy.misc, scipy.signal
import cv2
import sys
import os
import datetime
import natsort
import warnings
import math
warnings.filterwarnings('ignore')

def computeTextureWeights(fin, sigma, sharpness):
    dt0_v = np.vstack((np.diff(fin, n=1, axis=0), fin[0,:]-fin[-1,:]))
    dt0_h = np.vstack((np.diff(fin, n=1, axis=1).conj().T, fin[:,0].conj().T-fin[:,-1].conj().T)).conj().T

    gauker_h = scipy.signal.convolve2d(dt0_h, np.ones((1,sigma)), mode='same')
    gauker_v = scipy.signal.convolve2d(dt0_v, np.ones((sigma,1)), mode='same')

    W_h = 1/(np.abs(gauker_h)*np.abs(dt0_h)+sharpness)
    W_v = 1/(np.abs(gauker_v)*np.abs(dt0_v)+sharpness)

    return  W_h, W_v
    


In [34]:
def solveLinearEquation(IN, wx, wy, lamda):
    [r, c] = IN.shape
    k = r * c
    dx =  -lamda * wx.flatten('F')
    dy =  -lamda * wy.flatten('F')
    tempx = np.roll(wx, 1, axis=1)
    tempy = np.roll(wy, 1, axis=0)
    dxa = -lamda *tempx.flatten('F')
    dya = -lamda *tempy.flatten('F')
    tmp = wx[:,-1]
    tempx = np.concatenate((tmp[:,None], np.zeros((r,c-1))), axis=1)
    tmp = wy[-1,:]
    tempy = np.concatenate((tmp[None,:], np.zeros((r-1,c))), axis=0)
    dxd1 = -lamda * tempx.flatten('F')
    dyd1 = -lamda * tempy.flatten('F')
    
    wx[:,-1] = 0
    wy[-1,:] = 0
    dxd2 = -lamda * wx.flatten('F')
    dyd2 = -lamda * wy.flatten('F')
    
    Ax = scipy.sparse.spdiags(np.concatenate((dxd1[:,None], dxd2[:,None]), axis=1).T, np.array([-k+r,-r]), k, k)
    Ay = scipy.sparse.spdiags(np.concatenate((dyd1[None,:], dyd2[None,:]), axis=0), np.array([-r+1,-1]), k, k)
    D = 1 - ( dx + dy + dxa + dya)
    A = ((Ax+Ay) + (Ax+Ay).conj().T + scipy.sparse.spdiags(D, 0, k, k)).T
    
    tin = IN[:,:]
    tout = scipy.sparse.linalg.spsolve(A, tin.flatten('F'))
    OUT = np.reshape(tout, (r, c), order='F')
    
    return OUT
    


In [35]:
def tsmooth(img, lamda=0.01, sigma=3.0, sharpness=0.001):
    I = cv2.normalize(img.astype('float64'), None, 0.0, 1.0, cv2.NORM_MINMAX)
    x = np.copy(I)
    wx, wy = computeTextureWeights(x, sigma, sharpness)
    S = solveLinearEquation(I, wx, wy, lamda)
    return S

def rgb2gm(I):
    if (I.shape[2] == 3):
        I = cv2.normalize(I.astype('float64'), None, 0.0, 1.0, cv2.NORM_MINMAX)
        I = np.abs((I[:,:,0]*I[:,:,1]*I[:,:,2]))**(1/3)

    return I

def applyK(I, k, a=-0.3293, b=1.1258):
    f = lambda x: np.exp((1-x**a)*b)
    beta = f(k)
    gamma = k**a
    J = (I**gamma)*beta
    return J

def entropy(X):
    tmp = X * 255
    tmp[tmp > 255] = 255
    tmp[tmp<0] = 0
    tmp = tmp.astype(np.uint8)
    _, counts = np.unique(tmp, return_counts=True)
    pk = np.asarray(counts)
    pk = 1.0*pk / np.sum(pk, axis=0)
    S = -np.sum(pk * np.log2(pk), axis=0)
    return S

def maxEntropyEnhance(I, isBad, a=-0.3293, b=1.1258):
    # Esatimate k
    tmp = cv2.resize(I, (50,50), interpolation=cv2.INTER_AREA)
    tmp[tmp<0] = 0
    tmp = tmp.real
    Y = rgb2gm(tmp)
    
    isBad = isBad * 1
    isBad = scipy.misc.imresize(isBad, (50,50), interp='bicubic', mode='F')
    isBad[isBad<0.5] = 0
    isBad[isBad>=0.5] = 1
    Y = Y[isBad==1]
    
    if Y.size == 0:
       J = I
       return J
    
    f = lambda k: -entropy(applyK(Y, k))
    opt_k = scipy.optimize.fminbound(f, 1, 7)
    
    # Apply k
    J = applyK(I, opt_k, a, b) - 0.01
    return J


In [36]:
def exposer_fusion(img, mu=0.5, a=-0.3293, b=1.1258):
    lamda = 0.5
    sigma = 5
    I = cv2.normalize(img.astype('float64'), None, 0.0, 1.0, cv2.NORM_MINMAX)

    # Weight matrix estimation
    t_b = np.max(I, axis=2)
    t_our = cv2.resize(tsmooth(scipy.misc.imresize(t_b, 0.5, interp='bicubic', mode='F'), lamda, sigma), (t_b.shape[1], t_b.shape[0]), interpolation=cv2.INTER_AREA)    
    
    # Apply camera model with k(exposure ratio)
    isBad = t_our < 0.5
    J = maxEntropyEnhance(I, isBad)

    # W: Weight Matrix
    t = np.zeros((t_our.shape[0], t_our.shape[1], I.shape[2]))
    for i in range(I.shape[2]):
        t[:,:,i] = t_our
    W = t**mu
    
    #fusion
    I2 = I*W
    J2 = J*(1-W)

    result = I2 + J2
    result = result * 255
    result[result > 255] = 255
    result[result<0] = 0
    return result.astype(np.uint8)



In [37]:
def color_correction(r,u_r,u_ref,L2):
    L1 = np.max(r)
    gainFactor = L1 * (u_r/ u_ref) +L2
    Out = r / gainFactor
    return Out

def OptimalParameter(sceneRadiance):
    img = np.float64(sceneRadiance / 255)
    b, g, r = cv2.split(img)

    u_r = np.sum(r)
    u_g = np.sum(g)
    u_b = np.sum(b)
    u_ref = (u_r ** 2 + u_g ** 2 + u_b ** 2) ** 0.5
    L2 = 0.25
    r = color_correction(r, u_r, u_ref, L2)
    g = color_correction(g, u_g, u_ref, L2)
    b = color_correction(b, u_b, u_ref, L2)

    sceneRadiance = np.zeros((img.shape), 'float64')
    sceneRadiance[:, :, 0] = b
    sceneRadiance[:, :, 1] = g
    sceneRadiance[:, :, 2] = r
    sceneRadiance = sceneRadiance * 255
    sceneRadiance = np.clip(sceneRadiance,0, 255)
    sceneRadiance = np.uint8(sceneRadiance)
    return sceneRadiance


def simplest_cb(img, percent):
    out_channels = []
    channels = cv2.split(img)
    totalstop = channels[0].shape[0] * channels[0].shape[1] * percent / 200.0
    for channel in channels:
        bc = cv2.calcHist([channel], [0], None, [256], (0,256), accumulate=False)
        lv = np.searchsorted(np.cumsum(bc), totalstop)
        hv = 255-np.searchsorted(np.cumsum(bc[::-1]), totalstop)
        lut = np.array([0 if i < lv else (255 if i > hv else round(float(i-lv)/float(hv-lv)*255)) for i in np.arange(0, 256)], dtype="uint8")
        out_channels.append(cv2.LUT(channel, lut))
    return cv2.merge(out_channels)


In [38]:
import numpy as np
import numpy.matlib



class WBsRGB:
    def __init__(self, gamut_mapping=2, upgraded=0):

        if upgraded == 1:
            self.features = np.load('models/features+.npy') # training encoded features
            self.mappingFuncs = np.load('models/mappingFuncs+.npy') # mapping correction functions
            self.encoderWeights = np.load('models/encoderWeights+.npy') # weight matrix for histogram encoding
            self.encoderBias = np.load('models/encoderBias+.npy') # bias vector for histogram encoding
            self.K = 75  # K value for nearest neighbor searching---for the upgraded model, we found 75 is better
        else:
            self.features = np.load('models/features.npy')  # training encoded features
            self.mappingFuncs = np.load('models/mappingFuncs.npy')  # mapping correction functions
            self.encoderWeights = np.load('models/encoderWeights.npy')  # weight matrix for histogram encoding
            self.encoderBias = np.load('models/encoderBias.npy')  # bias vector for histogram encoding
            self.K = 25  # K value for nearest neighbor searching

        self.sigma = 0.25  # fall-off factor for KNN blending
        self.h = 60 # histogram bin width
        # our results reported with gamut_mapping=2, however gamut_mapping=1 gives more compelling results with
        # over-saturated examples
        self.gamut_mapping = gamut_mapping #options: =1 for scaling, =2 for clipping



In [39]:
def encode(self, hist):
        """ Generates a compacted feature of a given RGB-uv histogram tensor. """
        histR_reshaped = np.reshape(np.transpose(hist[:, :, 0]),
                       (1, int(hist.size / 3)), order="F")  # reshaped red layer of histogram
        histG_reshaped = np.reshape(np.transpose(hist[:, :, 1]),
                                    (1, int(hist.size / 3)), order="F")  # reshaped green layer of histogram
        histB_reshaped = np.reshape(np.transpose(hist[:, :, 2]),
                                    (1, int(hist.size / 3)), order="F")  # reshaped blue layer of histogram
        hist_reshaped = np.append(histR_reshaped,
                                  [histG_reshaped, histB_reshaped])  # reshaped histogram n * 3 (n = h*h)
        feature = np.dot(hist_reshaped - self.encoderBias.transpose(), self.encoderWeights)  # compute compacted histogram feature
        return feature


In [40]:
def rgbUVhist(self, I):
        """ Computes an RGB-uv histogram tensor. """
        sz = np.shape(I)  # get size of current image
        if sz[0] * sz[1] > 202500:  # if it is larger than 450*450
            factor = np.sqrt(202500 / (sz[0] * sz[1]))  # rescale factor
            newH = int(np.floor(sz[0] * factor))
            newW = int(np.floor(sz[1] * factor))
            I = cv2.resize(I, (newW, newH), interpolation=cv2.INTER_NEAREST)  # resize image
        II = I.reshape(int(I.size / 3), 3)  # n*3
        inds = np.where((II[:, 0] > 0) & (II[:, 1] > 0) & (II[:, 2] > 0))  # remove any zero pixels
        R = II[inds, 0]  # red channel
        G = II[inds, 1]  # green channel
        B = II[inds, 2]  # blue channel
        I_reshaped = np.concatenate((R, G, B), axis=0).transpose()  # reshaped image (wo zero values)
        eps = 6.4 / self.h
        A = np.arange(-3.2, 3.19, eps)  # dummy vector
        hist = np.zeros((A.size, A.size, 3))  # histogram will be stored here
        Iy = np.sqrt(np.power(I_reshaped[:, 0], 2) + np.power(I_reshaped[:, 1], 2) +
                     np.power(I_reshaped[:, 2], 2))  # intensity vector
        for i in range(3):  # for each histogram layer, do
            r = []  # excluded channels will be stored here
            for j in range(3):  # for each color channel do
                if j != i:  # if current color channel does not match current histogram layer,
                    r.append(j)  # exclude it
            Iu = np.log(I_reshaped[:, i] / I_reshaped[:, r[1]])  # current color channel / the first excluded channel
            Iv = np.log(I_reshaped[:, i] / I_reshaped[:, r[0]])  # current color channel / the second excluded channel
            diff_u = np.abs(np.matlib.repmat(Iu, np.size(A), 1).transpose() - np.matlib.repmat(A, np.size(Iu),
                                                                                               1))  # differences in u space
            diff_v = np.abs(np.matlib.repmat(Iv, np.size(A), 1).transpose() - np.matlib.repmat(A, np.size(Iv),
                                                                                               1))  # differences in v space
            diff_u[diff_u >= (eps / 2)] = 0  # do not count any pixel has difference beyond the threshold in the u space
            diff_u[diff_u != 0] = 1  # remaining pixels will be counted
            diff_v[diff_v >= (eps / 2)] = 0  # do not count any pixel has difference beyond the threshold in the v space
            diff_v[diff_v != 0] = 1  # remaining pixels will be counted
            # here, we will use a matrix multiplication expression to compute eq. 4 in the main paper.
            # why? because it is much faster
            temp = (np.matlib.repmat(Iy, np.size(A), 1) * (diff_u).transpose())  # Iy .* diff_u' (.* element-wise mult)
            hist[:, :, i] = np.dot(temp, diff_v)  # initialize current histogram layer with Iy .* diff' * diff_v
            norm_ = np.sum(hist[:, :, i], axis=None)  # compute sum of hist for normalization
            hist[:, :, i] = np.sqrt(hist[:, :, i] / norm_)  # (hist/norm)^(1/2)
        return hist


In [41]:
    def correctImage(self, I):
        """ White balance a given image I. """
        I = cv2.cvtColor(I, cv2.COLOR_BGR2RGB)  # convert from BGR to RGB
        I = im2double(I)  # convert to double
        feature = self.encode(self.rgbUVhist(I))
        D_sq = np.einsum('ij, ij ->i', self.features, self.features)[:, None] + \
               np.einsum('ij, ij ->i', feature, feature) - \
               2 * self.features.dot(feature.T)  # squared euclidean distances

        idH = D_sq.argpartition(self.K, axis=0)[:self.K]  # get smallest K distances
        mappingFuncs = np.squeeze(self.mappingFuncs[idH, :])
        dH = np.sqrt(
            np.take_along_axis(D_sq, idH, axis=0))  # square root nearest distances to get real euclidean distances
        sorted_idx = dH.argsort(axis=0)  # get sorting indices
        idH = np.take_along_axis(idH, sorted_idx, axis=0)  # sort distance indices
        dH = np.take_along_axis(dH, sorted_idx, axis=0)  # sort distances
        weightsH = np.exp(-(np.power(dH, 2)) /
                          (2 * np.power(self.sigma, 2)))  # compute blending weights
        weightsH = weightsH / sum(weightsH)  # normalize blending weights
        mf = sum(np.matlib.repmat(weightsH, 1, 33) *
                 mappingFuncs, 0)  # compute the mapping function
        mf = mf.reshape(11, 3, order="F")  # reshape it to be 9 * 3
        I_corr = self.colorCorrection(I, mf) # apply it!
        return I_corr

    def colorCorrection(self, input, m):
        """ Applies a mapping function m to a given input image. """
        sz = np.shape(input) # get size of input image
        I_reshaped = np.reshape(input,(int(input.size/3),3),
                                order="F") # reshape input to be n*3 (n: total number of pixels)
        kernel_out = kernelP(I_reshaped) # raise input image to a higher-dim space
        out = np.dot(kernel_out, m) # apply m to the input image after raising it the selected higher degree
        if self.gamut_mapping == 1:
            out = normScaling(I_reshaped, out) # scaling based on input image energy
        elif self.gamut_mapping == 2:
            out = outOfGamutClipping(out) # clip out-of-gamut pixels
        else:
            raise Exception('Wrong gamut_mapping value')
        out = out.reshape(sz[0], sz[1], sz[2], order="F")  # reshape output image back to the original image shape
        out = cv2.cvtColor(out.astype('float32'), cv2.COLOR_RGB2BGR)
        return out


In [42]:
def normScaling(I, I_corr):
    """ Scales each pixel based on original image energy. """
    norm_I_corr = np.sqrt(np.sum(np.power(I_corr, 2), 1))
    inds = norm_I_corr != 0
    norm_I_corr = norm_I_corr[inds]
    norm_I = np.sqrt(np.sum(np.power(I[inds, :],2), 1))
    I_corr[inds, :] = I_corr[inds, :]/np.tile(norm_I_corr[:, np.newaxis], 3) * \
                      np.tile(norm_I[:, np.newaxis], 3)
    return I_corr


def kernelP(I):
    """ Kernel function: kernel(r, g, b) -> (r,g,b,rg,rb,gb,r^2,g^2,b^2,rgb,1)
        Ref: Hong, et al., "A study of digital camera colorimetric characterization
         based on polynomial modeling." Color Research & Application, 2001. """
    return (np.transpose((I[:,0], I[:,1], I[:,2], I[:,0] * I[:,1], I[:,0] * I[:,2],
                          I[:,1] * I[:,2], I[:, 0] * I[:, 0], I[:, 1] * I[:, 1],
                          I[:, 2] * I[:, 2], I[:, 0] * I[:, 1] * I[:, 2],
                          np.repeat(1,np.shape(I)[0]))))


In [43]:

def outOfGamutClipping(I):
    """ Clips out-of-gamut pixels. """
    I[I > 1] = 1 # any pixel is higher than 1, clip it to 1
    I[I < 0] = 0 # any pixel is below 0, clip it to 0
    return I

def im2double(im):
    """ Returns a double image [0,1] of the uint8 im [0,255]. """
    return cv2.normalize(im.astype('float'), None, 0.0, 1.0, cv2.NORM_MINMAX)
import numpy as np
import numpy.matlib


In [44]:
class WBsRGB:
    def __init__(self, gamut_mapping=2, upgraded=0):

        if upgraded == 1:
            self.features = np.load('models/features+.npy') # training encoded features
            self.mappingFuncs = np.load('models/mappingFuncs+.npy') # mapping correction functions
            self.encoderWeights = np.load('models/encoderWeights+.npy') # weight matrix for histogram encoding
            self.encoderBias = np.load('models/encoderBias+.npy') # bias vector for histogram encoding
            self.K = 75  # K value for nearest neighbor searching---for the upgraded model, we found 75 is better
        else:
            self.features = np.load('models/features.npy')  # training encoded features
            self.mappingFuncs = np.load('models/mappingFuncs.npy')  # mapping correction functions
            self.encoderWeights = np.load('models/encoderWeights.npy')  # weight matrix for histogram encoding
            self.encoderBias = np.load('models/encoderBias.npy')  # bias vector for histogram encoding
            self.K = 25  # K value for nearest neighbor searching

        self.sigma = 0.25  # fall-off factor for KNN blending
        self.h = 60 # histogram bin width
        # our results reported with gamut_mapping=2, however gamut_mapping=1 gives more compelling results with
        # over-saturated examples
        self.gamut_mapping = gamut_mapping #options: =1 for scaling, =2 for clipping


In [45]:
def encode(self, hist):
        """ Generates a compacted feature of a given RGB-uv histogram tensor. """
        histR_reshaped = np.reshape(np.transpose(hist[:, :, 0]),
                       (1, int(hist.size / 3)), order="F")  # reshaped red layer of histogram
        histG_reshaped = np.reshape(np.transpose(hist[:, :, 1]),
                                    (1, int(hist.size / 3)), order="F")  # reshaped green layer of histogram
        histB_reshaped = np.reshape(np.transpose(hist[:, :, 2]),
                                    (1, int(hist.size / 3)), order="F")  # reshaped blue layer of histogram
        hist_reshaped = np.append(histR_reshaped,
                                  [histG_reshaped, histB_reshaped])  # reshaped histogram n * 3 (n = h*h)
        feature = np.dot(hist_reshaped - self.encoderBias.transpose(), self.encoderWeights)  # compute compacted histogram feature
        return feature


In [46]:
    def rgbUVhist(self, I):
        """ Computes an RGB-uv histogram tensor. """
        sz = np.shape(I)  # get size of current image
        if sz[0] * sz[1] > 202500:  # if it is larger than 450*450
            factor = np.sqrt(202500 / (sz[0] * sz[1]))  # rescale factor
            newH = int(np.floor(sz[0] * factor))
            newW = int(np.floor(sz[1] * factor))
            I = cv2.resize(I, (newW, newH), interpolation=cv2.INTER_NEAREST)  # resize image
        II = I.reshape(int(I.size / 3), 3)  # n*3
        inds = np.where((II[:, 0] > 0) & (II[:, 1] > 0) & (II[:, 2] > 0))  # remove any zero pixels
        R = II[inds, 0]  # red channel
        G = II[inds, 1]  # green channel
        B = II[inds, 2]  # blue channel
        I_reshaped = np.concatenate((R, G, B), axis=0).transpose()  # reshaped image (wo zero values)
        eps = 6.4 / self.h
        A = np.arange(-3.2, 3.19, eps)  # dummy vector
        hist = np.zeros((A.size, A.size, 3))  # histogram will be stored here
        Iy = np.sqrt(np.power(I_reshaped[:, 0], 2) + np.power(I_reshaped[:, 1], 2) +
                     np.power(I_reshaped[:, 2], 2))  # intensity vector
        for i in range(3):  # for each histogram layer, do
            r = []  # excluded channels will be stored here
            for j in range(3):  # for each color channel do
                if j != i:  # if current color channel does not match current histogram layer,
                    r.append(j)  # exclude it
            Iu = np.log(I_reshaped[:, i] / I_reshaped[:, r[1]])  # current color channel / the first excluded channel
            Iv = np.log(I_reshaped[:, i] / I_reshaped[:, r[0]])  # current color channel / the second excluded channel
            diff_u = np.abs(np.matlib.repmat(Iu, np.size(A), 1).transpose() - np.matlib.repmat(A, np.size(Iu),
                                                                                               1))  # differences in u space
            diff_v = np.abs(np.matlib.repmat(Iv, np.size(A), 1).transpose() - np.matlib.repmat(A, np.size(Iv),
                                                                                               1))  # differences in v space
            diff_u[diff_u >= (eps / 2)] = 0  # do not count any pixel has difference beyond the threshold in the u space
            diff_u[diff_u != 0] = 1  # remaining pixels will be counted
            diff_v[diff_v >= (eps / 2)] = 0  # do not count any pixel has difference beyond the threshold in the v space
            diff_v[diff_v != 0] = 1  # remaining pixels will be counted
            # here, we will use a matrix multiplication expression to compute eq. 4 in the main paper.
            # why? because it is much faster
            temp = (np.matlib.repmat(Iy, np.size(A), 1) * (diff_u).transpose())  # Iy .* diff_u' (.* element-wise mult)
            hist[:, :, i] = np.dot(temp, diff_v)  # initialize current histogram layer with Iy .* diff' * diff_v
            norm_ = np.sum(hist[:, :, i], axis=None)  # compute sum of hist for normalization
            hist[:, :, i] = np.sqrt(hist[:, :, i] / norm_)  # (hist/norm)^(1/2)
        return hist


In [47]:

    def correctImage(self, I):
        """ White balance a given image I. """
        I = cv2.cvtColor(I, cv2.COLOR_BGR2RGB)  # convert from BGR to RGB
        I = im2double(I)  # convert to double
        feature = self.encode(self.rgbUVhist(I))
        D_sq = np.einsum('ij, ij ->i', self.features, self.features)[:, None] + \
               np.einsum('ij, ij ->i', feature, feature) - \
               2 * self.features.dot(feature.T)  # squared euclidean distances

        idH = D_sq.argpartition(self.K, axis=0)[:self.K]  # get smallest K distances
        mappingFuncs = np.squeeze(self.mappingFuncs[idH, :])
        dH = np.sqrt(
            np.take_along_axis(D_sq, idH, axis=0))  # square root nearest distances to get real euclidean distances
        sorted_idx = dH.argsort(axis=0)  # get sorting indices
        idH = np.take_along_axis(idH, sorted_idx, axis=0)  # sort distance indices
        dH = np.take_along_axis(dH, sorted_idx, axis=0)  # sort distances
        weightsH = np.exp(-(np.power(dH, 2)) /
                          (2 * np.power(self.sigma, 2)))  # compute blending weights
        weightsH = weightsH / sum(weightsH)  # normalize blending weights
        mf = sum(np.matlib.repmat(weightsH, 1, 33) *
                 mappingFuncs, 0)  # compute the mapping function
        mf = mf.reshape(11, 3, order="F")  # reshape it to be 9 * 3
        I_corr = self.colorCorrection(I, mf) # apply it!
        return I_corr



In [48]:

    

def normScaling(I, I_corr):
    """ Scales each pixel based on original image energy. """
    norm_I_corr = np.sqrt(np.sum(np.power(I_corr, 2), 1))
    inds = norm_I_corr != 0
    norm_I_corr = norm_I_corr[inds]
    norm_I = np.sqrt(np.sum(np.power(I[inds, :],2), 1))
    I_corr[inds, :] = I_corr[inds, :]/np.tile(norm_I_corr[:, np.newaxis], 3) * \
                      np.tile(norm_I[:, np.newaxis], 3)
    return I_corr


def kernelP(I):
    """ Kernel function: kernel(r, g, b) -> (r,g,b,rg,rb,gb,r^2,g^2,b^2,rgb,1)
        Ref: Hong, et al., "A study of digital camera colorimetric characterization
         based on polynomial modeling." Color Research & Application, 2001. """
    return (np.transpose((I[:,0], I[:,1], I[:,2], I[:,0] * I[:,1], I[:,0] * I[:,2],
                          I[:,1] * I[:,2], I[:, 0] * I[:, 0], I[:, 1] * I[:, 1],
                          I[:, 2] * I[:, 2], I[:, 0] * I[:, 1] * I[:, 2],
                          np.repeat(1,np.shape(I)[0]))))

def outOfGamutClipping(I):
    """ Clips out-of-gamut pixels. """
    I[I > 1] = 1 # any pixel is higher than 1, clip it to 1
    I[I < 0] = 0 # any pixel is below 0, clip it to 0
    return I

def im2double(im):
    """ Returns a double image [0,1] of the uint8 im [0,255]. """
    return cv2.normalize(im.astype('float'), None, 0.0, 1.0, cv2.NORM_MINMAX)

